In [5]:
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `medgemma` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential 

Install required libraries (copy–paste)

In the first cell, paste this and run it:


In [1]:
!pip install -q transformers accelerate torch


Load MedGemma model

In [12]:
"google/gemma-2b-it"


'google/gemma-2b-it'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("✅ MedGemma loaded successfully")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ MedGemma loaded successfully


STEP 1️⃣ Lock ONE case (don’t touch dataset)

Use Case 2 from your table (the strong resistant one).

Case	Mutations	Resistance_Probability	Prediction(hiv_clinical_inference.ipynb)

0	Wild-type	None	0.000012	Susceptible

1	Single mutation (I84V)	I84V	0.000060	Susceptible

2	Multiple mutations	M46I+I54V+V82A+I84V+L90M	0.938000	Resistant--- using only this use case)


In [5]:
case_json = """
{
  "case_id": "Case_2",
  "case_type": "Multiple mutations",
  "gene": "HIV protease",
  "mutations": ["M46I", "I54V", "V82A", "I84V", "L90M"],
  "model_prediction": {
    "resistance_probability": 0.938,
    "predicted_label": "Resistant"
  },
  "confidence": "High",
  "limitations": [
    "Model trained mainly on subtype B data",
    "Treatment history not included"
  ],
  "disclaimer": "Decision support only, not a diagnosis"
}
"""


STEP 2️⃣ Clinician prompt (FIRST and MOST important)

Paste this exactly:

In [6]:
prompt_clinician = f"""
You are a clinical decision-support assistant.

The following JSON comes from a machine learning model predicting HIV drug resistance.
This output is NOT a diagnosis.

JSON:
{case_json}

Tasks:
1. Explain the resistance risk in clinician-friendly language
2. Identify which mutations most strongly drive the prediction
3. State confidence and limitations clearly
4. Mention when caution is required

Do NOT recommend specific drugs.
"""


STEP 3️⃣ Run MedGemma (actual reasoning)

In [7]:
inputs = tokenizer(prompt_clinician, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=250,
    do_sample=False
)

clinician_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(clinician_response)



You are a clinical decision-support assistant.

The following JSON comes from a machine learning model predicting HIV drug resistance.
This output is NOT a diagnosis.

JSON:

{
  "case_id": "Case_2",
  "case_type": "Multiple mutations",
  "gene": "HIV protease",
  "mutations": ["M46I", "I54V", "V82A", "I84V", "L90M"],
  "model_prediction": {
    "resistance_probability": 0.938,
    "predicted_label": "Resistant"
  },
  "confidence": "High",
  "limitations": [
    "Model trained mainly on subtype B data",
    "Treatment history not included"
  ],
  "disclaimer": "Decision support only, not a diagnosis"
}


Tasks:
1. Explain the resistance risk in clinician-friendly language
2. Identify which mutations most strongly drive the prediction
3. State confidence and limitations clearly
4. Mention when caution is required

Do NOT recommend specific drugs.
**Explanation:**

The model predicts a high probability of drug resistance (0.938) based on the presence of multiple mutations in the HIV pr

STEP 4️⃣ Patient-friendly explanation (human-centered part)

Now add one more prompt.

In [23]:
prompt_patient = f"""
You are explaining health information to a patient.

Context:
This information describes how well some HIV medicines may work against the virus.
It does NOT describe the patient’s genes, health status, prognosis, or treatment plan.

Facts you may use:
- A computer model analyzed patterns in the virus
- Some medicines may work less effectively
- Predictions are uncertain
- This is only one piece of information

Rules (MANDATORY):
- Use calm, reassuring language
- Do NOT mention genes, mutations, viral load, or codes
- Do NOT give advice or suggestions of any kind
- Do NOT suggest talking about changing medicines or options
- Do NOT imply worsening health or outcomes
- Avoid fear-inducing language
- Under 120 words

Task:
Explain what this result means for the patient.

The result suggests that some HIV medicines may not work as well against the virus in this situation.
This information comes from a computer model that looks at patterns in the virus, not from your body or overall health.

These predictions are not certain and are used as one piece of information.
Healthcare professionals review results like this together with other information to better understand what they may mean.






"""


this code is to generate response

In [24]:
inputs = tokenizer(prompt_patient, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=120,
    do_sample=False
)

patient_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(patient_response)



You are explaining health information to a patient.

Context:
This information describes how well some HIV medicines may work against the virus.
It does NOT describe the patient’s genes, health status, prognosis, or treatment plan.

Facts you may use:
- A computer model analyzed patterns in the virus
- Some medicines may work less effectively
- Predictions are uncertain
- This is only one piece of information

Rules (MANDATORY):
- Use calm, reassuring language
- Do NOT mention genes, mutations, viral load, or codes
- Do NOT give advice or suggestions of any kind
- Do NOT suggest talking about changing medicines or options
- Do NOT imply worsening health or outcomes
- Avoid fear-inducing language
- Under 120 words

Task:
Explain what this result means for the patient.

The result suggests that some HIV medicines may not work as well against the virus in this situation. 
This information comes from a computer model that looks at patterns in the virus, not from your body or overall healt

here we freeze the 1️⃣ Freeze the patient explanation as a constant

In [27]:
PATIENT_EXPLANATION_FINAL = """
The result suggests that some HIV medicines may not work as well against the virus in this situation. This information comes from a computer model that analyzes patterns in the virus sample, rather than information about your overall health.

These predictions are not certain and are used as one piece of information. Healthcare professionals review results like this together with other information to better understand what they may mean.
"""


2️⃣ Replace generation code with a simple print

In [28]:
print(PATIENT_EXPLANATION_FINAL)



The result suggests that some HIV medicines may not work as well against the virus in this situation. This information comes from a computer model that analyzes patterns in the virus sample, rather than information about your overall health.

These predictions are not certain and are used as one piece of information. Healthcare professionals review results like this together with other information to better understand what they may mean.

